In [150]:
initial_spaces = [['w']*8, ['w']*8, ['']*8, ['']*8, ['']*8, ['']*8, ['b']*8, ['b']*8]
empty_board = [['']*8 for _ in range(8)] 
#empty_board = [[0]*8]*8
        
class chessboard(object):
    
    def __init__(self, spaces = initial_spaces):
        for i in range(len(spaces)):
            for j in range(len(spaces)):
                empty_board[j][i] = spaces[i][j] #want the board to be properly oriented
        self.spaces = empty_board
        
    def isOccupied(self, position, direction):
        x, y = position
        new_x, new_y = x + direction[0], y + direction[1]
        return bool(board.spaces[new_x][new_y])
    
board = chessboard()

In [152]:
board.spaces

[['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b']]

Some ideas:
-the chessboard class might be redundant. Can probably accomplish the same task using the configuration class
-create abstract class chessPiece(object) with functions isOccupied and northIsClear (for rooks/queens) eastIsClear, diagonalIsClear etc. all other classes can inherit these methods
-create vision method in the abstract class

Utilize commands and lambda functions to reduce the amount of branching?

How to accomplish castling??

In [26]:
board.isOccupied((0, 0), (0, 6))

True

In [215]:
class chessPiece(object):
    
    def __init__(self, position, is_white, alive = True): #position must be a tuple
        self.position = position
        self.alive = alive
        self.is_white = is_white
        
    def move(self, direction):
        if not self.alive:
            print('That piece has already been taken. Choose a different piece!')
        x, y = self.position
        new_x, new_y = x + direction[0], y + direction[1]
        if any([new_x < 0, new_x > 7, new_y < 0, new_y > 7]):
            print('Not a valid move. That would place your piece off the board!')
        elif (board.spaces[new_x][new_y] == 'w' and self.is_white) or (board.spaces[new_x][new_y] == 'b' and not self.is_white):
            print('Not a valid move. That space is occupied by your own piece!')
        else:
            return (new_x, new_y)
        
    def do_move(self, direction):
        x , y = self.position
        new_x, new_y = x + direction[0], y + direction[1]
        new_pos = (new_x, new_y)
        if self.is_white:
                board.spaces[new_x][new_y] = 'w'
                board.spaces[x][y] = ''
        else:
            board.spaces[new_x][new_y] = 'b'
            board.spaces[x][y] = ''
        return new_pos

In [191]:
class whitePawn(chessPiece):
    valid_moves = {(0, 1), (0, 2), (1, 1), (-1, 1)}
    def __init__(self, position, alive = True, is_white = True, has_moved = False):
        super().__init__(position, alive, is_white)
        self.has_moved = has_moved
        
    def whitePawnMove(self, direction):
        x, y = self.position #gonna need this at the last step
        new_x, new_y= x + direction[0], y + direction[1]
        new_pos = super().move(direction) 
        if direction not in self.valid_moves:
            print('That is not a valid move for a pawn! Choose a different move.')
        elif direction in {(1, 1), (-1, 1)} and not board.isOccupied(self.position, direction):
            print('There\'s no black piece on that tile. You can\'t attack.')
        #note that the .move() method in the parent class will detect if any attempted move would place the pawn
        #on a tile occupied by another white piece
        elif direction == (0, 2) and not self.has_moved:
            print('This pawn has already moved.')
        elif direction in {(0, 1), (0, 2)} and board.isOccupied(self.position, direction):
            print('That space is already occupied.')
        else:
            new_pos = super().move(direction)            #still need to add a .kill() method
            board.spaces[new_x][new_y] = 'w'
            print(new_x, new_y)
            board.spaces[x][y] = ''
            self.position = new_pos            #at a later stage I might need to revisit this and add a return command
            self.has_moved = True

In [200]:
class king(chessPiece):
    valid_moves = {(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)}
    def __init__(self, position, is_white, alive = True):
        super().__init__(position, alive, is_white)
        
    def is_in_check(self):
        return False                      #placeholder
    
    def kingMove(self, direction):
        x, y = self.position
        new_x, new_y = x + direction[0], y + direction[1]
        new_pos = super().move(direction) 
        if direction not in self.valid_moves:
            print('Not a valid move for a king.')
        elif king(new_pos, self.is_white, alive = True).is_in_check():
            print('That move would put you in check!')
        else:
            if self.is_white:
                board.spaces[new_x][new_y] = 'w'
                board.spaces[x][y] = ''
            else:
                board.spaces[new_x][new_y] = 'b'
                board.spaces[x][y] = ''
            self.position = new_pos

In [ ]:
class knight(chessPiece):
    valid_moves = {[1, 2], [2, 1], [-1, 2], [2, -1], [1, -2], [-2, 1], [-1, -2], [-2, -1]}
    def __init__(self, position, is_white, alive = True):
        super().__init__(position, alive, is_white)
        
    def knightMove(self, direction):
        x, y = position
        new_x, new_y = x + direction[0], y + direction[1]
        new_pos = super().move(direction) 
        if direction not in self.valid_moves:
            print('Not a valid move for a knight')
        else:
            if self.is_white:
                board.spaces[new_x][new_y] = 'w'
                board.spaces[x][y] = ''
            else:
                board.spaces[new_x][new_y] = 'b'
                board.spaces[x][y] = ''
            self.position = new_pos

In [229]:
class bishop(chessPiece):
    valid_moves = [(a, a) for a in range(1, 8)] + [(-b, -b) for b in range(1, 8)] + [(-c, c) for c in range(1, 8)] + [(d, -d) for d in range(1, 8)]
    
    def __init__(self, position, is_white, alive = True):
        super().__init__(position, alive, is_white)
        
    def bishopMove(self, direction):
        new_pos = super().move(direction)
        if direction not in self.valid_moves:
            print('Not a valid move for a bishop')
        else:
            self.position = self.do_move(direction)    

In [230]:
board = chessboard()
b1 = bishop((1, 3), True)
b1.bishopMove((1,-1))
b1.position

(2, 2)

In [225]:
board.spaces

[['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', '', '', '', '', '', 'b', 'b'],
 ['w', 'w', 'w', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b']]

In [207]:
k1 = king((4, 0), is_white = True)
k1.kingMove((0, -1))
k1.position

Not a valid move. That would place your piece off the board!


In [204]:
wp1 = whitePawn((4, 1))
wp1.whitePawnMove((0, 1))
wp1.position

4 2


(4, 2)

In [208]:
board.spaces

[['w', '', 'w', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', '', 'w', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['', 'w', 'w', '', '', '', 'b', 'w'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b']]

In [ ]:
class whiteKing(object):
        
    def __init__(self, position, is_alive = True, is_white = True): #position must be a tuple
        self.position = position 
        self.alive = alive
        self.has_moved = has_moved
    
    def move(self, direction): #direction must be a tuple
        valid_directions = {(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)}
        x, y = self.position
        new_x, new_y = x + direction[0], y + direction[1]
        
        if direction not in valid_directions:
            return 'Not a valid move for a king.'
        elif
        
        else:
            self.position = (new_x, new_y)

In [7]:
board = chessboard()
board.spaces

[['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b'],
 ['w', 'w', '', '', '', '', 'b', 'b']]

In [ ]:
board = chessboard(initial_spaces)
board.spaces

In [ ]:
for i in range(len(initial_spaces)):
    for j in range(len(initial_spaces)):
        empty_board[j][i] = initial_spaces[i][j] #want the board to be properly oriented

In [ ]:
empty_board[0][2]

In [ ]:
board.spaces[0][2]